In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.timeseries import aggregate_downsample
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import LombScargle
from astropy.table import Table
from astropy.time import Time
from astropy.timeseries import TimeSeries
from astropy.utils.data import get_pkg_data_filename

In [ ]:
"""
"""
def boundify(a, sigma=2.0):
    mean = a.mean()
    dev = np.std(a)
    
    return (mean - (sigma * dev)).value, (mean + (sigma * dev)).value

In [ ]:
filename = get_pkg_data_filename('/data/SuperWASP/1SWASP J144508.70+050514.4.fits')

In [ ]:
t = Table.read(filename, hdu=1)
# adjust timestamp to day units
t['TMID'] = (t['TMID'] / 86400.0) + 2453005.5
t['TMID'].unit = 'day'
t['TAMFLUX2'].unit = 'mag'
t['TAMFLUX2_ERR'].unit = 'mag'
t['time'] = Time(t['TMID'], format='jd')
t.keep_columns(['time', 'TAMFLUX2', 'TAMFLUX2_ERR'])

In [ ]:
ts = TimeSeries(t)

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(ts.time.jd, ts['TAMFLUX2'], 'k.', markersize=1)
plt.xlabel('Julian Date')
plt.ylabel('Flux (micro vega)')

In [ ]:
min_period = 2.0 * u.hour
max_period = 12.0 * u.hour

periodogram = LombScargle.from_timeseries(
    ts,
    signal_column_name='TAMFLUX2',
    uncertainty='TAMFLUX2_ERR',
    nterms=6
)

frequency, power = periodogram.autopower(
    nyquist_factor=5,
    minimum_frequency=1.0 / (max_period.to(u.day)),
    maximum_frequency=1.0 / (min_period.to(u.day)),
    samples_per_peak=10
)

best = power.argmax()
period = 1.0 / frequency[best]

plt.plot(frequency, power)

In [ ]:
# sort the power series from best to worst; this allows us to quickly try alternatives
# alternatively just use power.argmax()
sorted = power.argsort()[::-1]
# convert to frequency
period = 1.0 / frequency[sorted[0]]

In [ ]:
# fold the lightcurve according to the selected period
ts_folded = ts.fold(period=period, normalize_phase=False, wrap_phase=period)
period.to(u.hour)

In [ ]:
# plot the folded light-curve (single-phase)
y_min, y_max = boundify(ts_folded['TAMFLUX2'])

plt.figure(figsize=(20,20))
plt.plot(ts_folded.time.jd, ts_folded['TAMFLUX2'], 'k.', markersize=1)
plt.ylim(bottom=y_min, top=y_max)
plt.xlabel('Phase')
plt.ylabel('SAP Flux (e-/s)')

In [ ]:
# calculate the mean
mean, median, stddev = sigma_clipped_stats(ts_folded['TAMFLUX2'])
ts_folded['FLUX2_NORM'] = ts_folded['TAMFLUX2'] #/ median
ts_binned = aggregate_downsample(ts_folded, time_bin_size=0.001 * u.day)

In [ ]:
# now chart the light-curve
y_min, y_max = boundify(ts_folded['FLUX2_NORM'])

plt.figure(figsize=(20,20))
# draw 2 phases of the base light-curve
plt.plot((ts_folded.time.jd / period.to(u.day)).value, ts_folded['FLUX2_NORM'], 'k.', markersize=1)
plt.plot((ts_folded.time.jd / period.to(u.day)).value - 1.0, ts_folded['FLUX2_NORM'], 'k.', markersize=1)
# draw 2 phases of the mean 
plt.plot((ts_binned.time_bin_start.jd / period.to(u.day)).value, ts_binned['FLUX2_NORM'], 'r-', drawstyle='steps', markersize=1)
plt.plot((ts_binned.time_bin_start.jd / period.to(u.day)).value - 1.0, ts_binned['FLUX2_NORM'], 'r-', drawstyle='steps', markersize=1)
plt.ylim(bottom=y_min, top=y_max)
plt.xlabel('Phase')
plt.ylabel('Flux (micro vega)')